<a href="https://colab.research.google.com/github/mapzine123/machine-running/blob/main/%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 정형 데이터와 비정형 데이터
정형 데이터 : 어떤 구조로 되어 있다는 뜻.<br>
ex) 엑셀, csv, DB등<br>
<br>
비정형 데이터 : 엑셀, DB 등으로 표현하기 어려운 것들<br>
ex) 텍스트 데이터, 사진, 음악 등<br>
<br>

선형 회귀, 릿지, 라쏘, 다항회귀, 로지스틱 회귀 등은 정형 데이터에 잘 맞다.<br>
정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘은 ***앙상블 학습***이라는 거다.
<br>
<br>
그럼 비정형 데이터에는 어떤 알고리즘을 사용해야 할까?<br>
***신경망 알고리즘***을 사용한다.<br>
비정형 데이터는 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 까다롭다.

# 랜덤 포레스트
앙상블 학습의 대표 주자 중 하나.<br>
안정적인 성능 덕에 널리 사용됨<br>
랜덤하게 선택한 샘플과 특성을 사용한다. 때문에,<br>
1. 훈련 세트에 과대적합 되는 것을 막아준다.<br>
2. 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.<br>

<br>
결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만드는 것<br>
각 결정 트리의 예측을 사용해 최종 예측을 만든다.<br>

각 트리를 훈련하기 위한 데이터를 랜덤으로 만든다.<br>
훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만드는데, 이때 한 샘플이 중복되어 추출될 수 있다.<br>
<br>
ex) 1000개의 가방에서 100개쌕 샘플을 뽑으면, 먼저 1개를 뽑고, 뽑았던걸 다시 가방에 넣음. 이런식으로 100개를 가방에서 뽑으면 중복된 샘플을 뽑을 수 있음<br>
이렇게 만들어진 샘플을 ***부트스트랩 샘플*** 이라고 부름<br>

부트 스트랩이란?<br>
- 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식을 의미<br>
<br>


In [1]:
# 데이터 준비
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
# cross_validate를 이용한 교차 검증 숳애
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


훈련 세트에 과대적합 된 것 같음.

In [3]:
# 랜덤 포레스트 모델의 훈련세트에 훈련한 후 특성 중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


[0.12345626 0.86862934 0.0079144]<br>
'결정 트리'에서 만든 특성 중요도<br>
각각 [alcohol, sugar, pH]를 나타냄<br>


---


sugar의 중요도가 줄고, pH와 alcohol의 중요도가 상승함<br>
랜덤 포레스트는 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문에<br>
하나의 특성에 과도하게 집중하지 않고, 좀 더 많은 특성이 훈련에 기여할 기회를 얻음<br>
이는 과대적합을 줄이고, 일반화 성능을 높이는 데 도움이 됨



---

RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있음<br>
훈련 세트에서 중복을 허용하여 부트스트램 샘플을 만들어서 결정 트리를 훈련하는데,<br>
이때 부트스트램 샘플에 포함되지 않고 남는 샘플이 있음.<br>
이런 샘플을 ***OOB*** (Out Of Bag) 이라고 함<br>
이 남는 샘플을 이용해 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있음<br>
마치 검증 세트의 역할을 수행함<br>

In [4]:
# OOB 점수 출력하기
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리


- 부트스트랩 샘플을 사용하지 않음 (결정 트리를 만들 때 전체 훈련 세트를 사용함)
- 데이터 세트에서 중복을 허용하지 않고 데이터를 샘플링 함
- 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라, 무작위로 분할함
- 하나의 결정 트리에서 특성을 무작위로 분할하면 성능이 낮아지겠지만, 트리를 앙상블 하기 때문에 과대 적합을 막고 검증 세트의 점수를 높이는 효과가 있음

In [7]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [8]:
# 엑스트라 트리 특성 중요도
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그레이디언트 부스팅 (Gradient boosting)
- 깊이가 얕은 결정 트리를 사용해 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
- sklearn의 GradientBoostingClassifier은 깊이가 3인 결정트리를 100개 사용
- 깊이가 얕은 결정 트리를 사용하기 때문에, 과대 적합에 강하고 옾은 일반화 성능을 기대할 수 있음
- Gradient라는 이름에 맞게, 경사 하강법을 사용하여 트리를 앙상블에 추가함
- 분류에서는 로지스틱 손실 함수, 회귀에서는 평균 제곱 오차 함수 사용
- 결정 트리를 게속 추가하면서 가장 낮은 곳을 찾아 이동
- 경사 하상법처럼 낮은곳으로 천천히 조금씩 이동 (학습률 매개변수로 속도 조절)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [14]:
# 학습률을 증가시키고 트리의 개수를 늘리면?
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


그레이디언트 부스팅이 랜덤 포레스트보다 일부 특성(당도)에 더 집중한다.

In [15]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있음<br>
하지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느림

#히스토그램 기반 그레이디언트 부스팅 (Histogram-based Gradient Boosting)
- 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘
- 입력 특성을 256개 구간으로 나눔 -> 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
- 256개 구간 중 하나를 떼어 놓고, 누락된 값을 위해 사용함 -> 입력에 누락된 특성이 있어도 따로 전처리를 할 필요가 없음


In [16]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


과대 적합을 잘 억제하며 그레이디언트 부스팅보다 조금 더 높은 성능을 제공

In [18]:
# 특성 중요도 계산
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)

# 특성을 하나씩 랜덤하게 섞어 모델의 성능이 변화하였느지를 관찰하여 어떤 특성이 중요한지를 계산하는 함수
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance() 함수는 importances(특성 중요도), importances_mean(평균), importances_std(표준 편차)를 담고 있음

In [19]:
# test 세트에서 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


[알코올 도수, 당도, pH순]<br>
당도에 집중하는 것을 알 수 있음

In [20]:
# 최종 성능 확인
hgb.score(test_input, test_target)

0.8723076923076923

# 여러가지 HistGradientBoosting 라이브러리
- XGBosst

In [21]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


- LightGBM

In [22]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


# 정리
- 앙상블 학습 : 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘

- 랜덤 포레스트 : 부트스트랩 샘플 사용, 대표 앙상블 학습 알고리즘
- 엑스트라 트리 : 부트스트랩 사용 안함, 결정 트리의 노드를 랜덤하게 분할
- 그레이디언트 부스팅 : 이전 트리의 손실을 보완하는 식으로 얕은 결정트리를 연속으로 추가
- 히스토그램 기반 그레이디언트 부스팅 : 훈련 데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄, 가장 인기있음